In [110]:
import pandas as pd
import numpy as np

# Problem 2: a linguistic street map of Singapore

There is considerable linguistic variety in Singapore road names: Malaysians names (Jalan Besar), British names (Northumberland road), Chinese names (Keong Saik Road), Indian names (Veerasamy road), Jewish names (Belilios road), and the usual generic sorts of names that describe either area landmarks or other common noun.

![Image](streets.png)

In [111]:
# load the dataset
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/streets.csv'
street_names = pd.read_csv(url)
street_names.head(10)

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

**``Origin`` column values:**

- Chinese (all dialects including Cantonese, Hokkien, Mandarin, etc)
- Malay
- Indian (all languages of the subcontinent)
- British
- Generic (Race Course Road, Sunrise Place, etc)
- Other (Other languages).

In [112]:
# number of words
name = 'Hoot Kiam Plume'
def get_number_of_words(name):
    return len(name.split(' '))

# avg word length
def get_avg_word_length(name):
    words = name.split(' ')
    return np.mean([len(word) for word in words])

# idk
def is_Malay(tag):
    return tag in ['Jalan', 'Lorong', 'Bukit', 'Lengkok', 'Taman', 'Kampong', 'Lengkong']
maylayTag = 'Jalan'
is_Malay(maylayTag)



True

In [113]:
street_names['number_of_words'] = street_names.name.apply(get_number_of_words)
street_names['avg_word_length'] = street_names.name.apply(get_avg_word_length)
street_names['is_malay'] = street_names.tag.apply(is_Malay)

In [114]:
street_names.head(1)

,name,tag,origin,number_of_words,avg_word_length,is_malay
0,Saiboo,Street,Indian,1,6.0,False


In [119]:
X = street_names.drop(['tag', 'origin'], axis=1)
y = street_names.origin


In [120]:
num_features = ['number_of_words', 'avg_word_length', 'is_malay']
text_features = ['name']

In [121]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [122]:
vect = CountVectorizer(analyzer='char', ngram_range=(1,4))

In [124]:
vect.fit_transform(X.name)

<2834x8262 sparse matrix of type '<class 'numpy.int64'>'
	with 64485 stored elements in Compressed Sparse Row format>

In [12]:
vect.get_feature_names_out()

array([' ', ' a', ' ab', ..., 'zubi', 'zuk', 'zuki'], dtype=object)

In [138]:
street_names.origin.value_counts()

Malay      927
British    798
Generic    497
Chinese    403
Other      167
Indian      42
Name: origin, dtype: int64

The **goal** is to predict the ``origin`` column from the ``street`` and ``tag`` columns.

**Part 1:** Add the following features:

- Number of words in road name (more words => more likely to be Chinese.) Assign it to a column named `n_words`.
- Average word length in road name (longer words => more likely to be British or Indian.) Assign it to a column named `avg_word_len`.
- Is the road tag Malay? (if yes => very correlated with being Malay.) **Malay tags**: *Jalan*, *Lorong*, *Bukit*, *Lengkok*, *Taman*, *Kampong*, *Lengkong*. Assign it to a column named `is_malay`.

**Part 2:** define the matrix X (columns `name`, `n_words`, `avg_word_len`, `is_malay`) and the target vector y (column `origin`)

In [125]:
street_names

,name,tag,origin,number_of_words,avg_word_length,is_malay
0,Saiboo,Street,Indian,1,6.0,False
1,Merchant,Loop,Generic,1,8.0,False
2,Hill,Street,Generic,1,4.0,False
3,Ophir,Road,Malay,1,5.0,False
4,Buona,Vista Road,Other,1,5.0,False
...,...,...,...,...,...,...
2829,Florence,Close,Chinese,1,8.0,False
2830,Hoot Kiam,Road,Chinese,2,4.0,False
2831,Clarke,Quay,British,1,6.0,False
2832,Countryside,Walk,Generic,1,11.0,False


In [132]:
X = street_names.drop(['tag', 'origin'], axis=1)
y = street_names.origin

In [172]:
X.head(3)

,name,number_of_words,avg_word_length,is_malay
0,Saiboo,1,6.0,False
1,Merchant,1,8.0,False
2,Hill,1,4.0,False


**Part 3:** split X and y into training and testing sets.

In [214]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.5)

**Part 4:** build a classification pipeline.

The pipeline must use `CountVectorizer` to compute the number of **character 1-gram, 2-grams, 3-grams and 4-grams** from the column `name` ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) has an `analyzer` parameter that you'll have to set to `analyzer='char'`.)

In [215]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1417, 4) (1417, 4) (1417,) (1417,)


In [248]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix


pipe = Pipeline(steps=[
    ('vectorizer', CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=1000)),
    ('clf', MultinomialNB())
])

# fit pipe
pipe.fit(X_train.name, y_train)


Pipeline(steps=[('vectorizer',
                 CountVectorizer(analyzer='char', max_features=1000,
                                 ngram_range=(1, 4))),
                ('clf', MultinomialNB())])

**Part 5:** Fit the classification pipeline to the training data, and  evaluate its performance on the testing set.

## NOTICE:
I'm not sure why y_test_pred is only coming up with 4 predictions.

In [249]:
y_test_pred = pipe.predict(X_test)
print(y_test_pred)
pipe.score(y_test_pred, y_test)

['Malay' 'British' 'British' 'Malay']


ValueError: Found input variables with inconsistent numbers of samples: [1417, 4]